# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
%matplotlib inline
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# Import csv data and create a dataframe.
cities_df = pd.read_csv('../WeatherPy/data/city_weather.csv')
# Drop the NA that were inserted for cities without weather info
cities_df.dropna(inplace=True)

In [3]:
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Manokwari,-0.866667,134.083333,85.06,64.0,72.0,5.10,ID,2020-09-16 03:34:46
1,New Norfolk,-42.780750,147.059387,64.40,34.0,90.0,18.34,AU,2020-09-16 03:34:46
2,Avarua,-21.207778,-159.775000,75.20,73.0,59.0,14.99,CK,2020-09-16 03:34:46
3,Chokurdakh,70.633333,147.916667,41.47,66.0,98.0,7.61,RU,2020-09-16 03:34:46
4,Hilo,19.729722,-155.090000,80.60,65.0,20.0,6.93,US,2020-09-16 03:34:47
...,...,...,...,...,...,...,...,...,...
995,Bar Harbor,44.387500,-68.204444,54.00,87.0,90.0,3.83,US,2020-09-16 03:38:12
996,Bogorodskoye,57.046945,41.013539,40.87,97.0,3.0,5.10,RU,2020-09-16 03:37:02
997,Kaihua,23.368701,104.279996,76.37,71.0,90.0,5.61,CN,2020-09-16 03:38:12
998,Chunhuhub,19.550000,-88.683333,76.35,98.0,100.0,1.86,MX,2020-09-16 03:38:12


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
coordinates = cities_df[['Lat','Lng']]

# store the figure in a variable so we can continue to add layers to the same map
fig = gmaps.figure()


# Store 'Poverty Rate' in a variable called 'poverty_rate'
humidity_data = cities_df['Humidity'].astype(float)

# Create the heatmap layer using locations and poverty_rate
heat_layer = gmaps.heatmap_layer(coordinates, weights=humidity_data, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=0.9)
# Add heatmap layer to figure
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.

weather_criteria_df = cities_df.loc[(cities_df['Max Temp'] < 80) & (cities_df['Max Temp'] > 70) & 
              (cities_df['Wind Speed'] < 10) & 
             (cities_df['Cloudiness'] == 0)]

print(f"The list of cities matching the criteria contains {len(weather_criteria_df)} cities.")
weather_criteria_df.reset_index(drop=True, inplace=True)
weather_criteria_df

The list of cities matching the criteria contains 29 cities.


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Konispol,39.658889,20.181389,75.99,72.0,0.0,7.65,AL,2020-09-16 03:34:47
1,Rikitea,-23.120278,-134.969167,74.03,72.0,0.0,1.59,PF,2020-09-16 03:34:49
2,Viransehir,37.235278,39.763056,78.44,18.0,0.0,4.05,TR,2020-09-16 03:34:56
3,Caravelas,-17.750000,-39.250000,70.77,94.0,0.0,4.65,BR,2020-09-16 03:35:00
4,Kedougou,12.483333,-11.950000,74.75,93.0,0.0,6.13,SN,2020-09-16 03:35:09
5,Gat,24.964722,10.172778,78.01,76.0,0.0,3.33,LY,2020-09-16 03:31:04
6,Saveh,35.021300,50.356600,70.34,34.0,0.0,2.44,IR,2020-09-16 03:35:30
7,Shu,43.598333,73.761389,77.07,19.0,0.0,8.14,KZ,2020-09-16 03:35:31
8,Camapua,-19.500000,-54.083333,75.07,26.0,0.0,5.57,BR,2020-09-16 03:35:57
9,Suez,29.966667,32.550000,73.65,87.0,0.0,8.86,EG,2020-09-16 03:36:05


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = weather_criteria_df

#### Setup the framework for the API call

* We are going to narrow the search for the first 10 cities.

In [7]:
# params dictionary to update each iteration
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Use the lat/lng we recovered to identify airports
for index, row in weather_criteria_df.iloc[0:10,:].iterrows():
    
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    print(f"Searching within {row['City']}, {row['Country']}...")

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)


    # convert to json
    name_address = name_address.json()

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Searching within Konispol, AL...


/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/gabrielpivaro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Searching within Rikitea, PF...
Searching within Viransehir, TR...
Searching within Caravelas, BR...
Searching within Kedougou, SN...
Missing field/result... skipping.
Searching within Gat, LY...
Searching within Saveh, IR...
Searching within Shu, KZ...
Searching within Camapua, BR...
Searching within Suez, EG...


In [8]:
# Drop the rows that contains NA leaving only the 10 hotels in the list with the desired columns
hotel_df = hotel_df[['City', 'Country', 'Lat', 'Lng', 'Hotel Name']].dropna()
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,Konispol,AL,39.658889,20.181389,Hotel Konispoli
1,Rikitea,PF,-23.120278,-134.969167,Pension Maro'i
2,Viransehir,TR,37.235278,39.763056,Viranşehir Öğretmenevi
3,Caravelas,BR,-17.750000,-39.250000,Pousada dos Navegantes
5,Gat,LY,24.964722,10.172778,مصطفى محمد صالح صابر شربكي
6,Saveh,IR,35.021300,50.356600,مجید کریمی
7,Shu,KZ,43.598333,73.761389,Shyngyzkhan
8,Camapua,BR,-19.500000,-54.083333,Fazenda Essência do Campo - Camapuã/MS
9,Suez,EG,29.966667,32.550000,Green House


In [9]:
# Export the hotels dataframe as csv
hotel_df.to_csv('data/hotels_list.csv',index=False)

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))